In [12]:
import uuid
import random

from tqdm import tqdm
from threading import Thread

from cassandra.cluster import Cluster, BatchStatement
from cassandra.policies import RetryPolicy, ExponentialReconnectionPolicy

from utils.query_utils import *
from cassandra import ConsistencyLevel
from cassandra.cluster import ExecutionProfile, EXEC_PROFILE_DEFAULT

from cassandra.cluster import Cluster
from cassandra import ConsistencyLevel
from cassandra.cluster import Cluster, ExecutionProfile, EXEC_PROFILE_DEFAULT
from cassandra.policies import WhiteListRoundRobinPolicy, DowngradingConsistencyRetryPolicy
from cassandra.query import tuple_factory

In [13]:
profile = ExecutionProfile(
    retry_policy = DowngradingConsistencyRetryPolicy,
    consistency_level = ConsistencyLevel.ALL,
    serial_consistency_level = ConsistencyLevel.LOCAL_SERIAL,
    request_timeout = 200,
)

In [14]:
CLUSTER_IDS = ['172.22.0.2']
KEYSPACE = 'library_keyspace'

cluster = Cluster(CLUSTER_IDS, execution_profiles={EXEC_PROFILE_DEFAULT: profile})
session = cluster.connect()
session.set_keyspace(KEYSPACE)

In [15]:
res = get_all_reservations(session)
for r in res:
    print(r.reservation_id, r.user_id, r.user_name, r.book_name)
    reservation_id = r.reservation_id
    
reservation_id

8dc4ea13-79e1-477a-839c-b0e7ccbca112 cf10d511-fdc7-43fd-8f46-cad92b8377a9 User 1 Book 1857
1b4d5297-f758-477c-9ec3-19488336790b b30070c5-44f6-467a-a4f0-781e98ae45c1 User 2 Book 1308
bd5d4ebf-f97a-4731-8f79-460a1639bb0c b30070c5-44f6-467a-a4f0-781e98ae45c1 User 2 Book 1819
7fabf1fb-3f25-4263-8b59-61af1cb15a04 cf10d511-fdc7-43fd-8f46-cad92b8377a9 User 1 Book 369
ecfbcb3d-1696-4bf2-a103-0b96e8077c32 b30070c5-44f6-467a-a4f0-781e98ae45c1 User 2 Book 437
9b3c2b62-6af3-4525-b22a-3f78960c8641 b30070c5-44f6-467a-a4f0-781e98ae45c1 User 2 Book 1929
a0f656b8-9ad7-47db-af4b-6ca060fe109d b30070c5-44f6-467a-a4f0-781e98ae45c1 User 2 Book 861
7bb162c7-4d1e-4313-960f-f81bccb304d0 cf10d511-fdc7-43fd-8f46-cad92b8377a9 User 1 Book 475
5ce98c3f-5b63-4f3e-b8d2-fca06b239473 cf10d511-fdc7-43fd-8f46-cad92b8377a9 User 1 Book 1942
d4d86495-451e-40b6-a544-09cbfc364474 b30070c5-44f6-467a-a4f0-781e98ae45c1 User 2 Book 940
de200858-69e0-416a-871d-2b06ea01a681 cf10d511-fdc7-43fd-8f46-cad92b8377a9 User 1 Book 1866
6232

UUID('aa834988-0c89-463c-b982-9e86bb038334')

In [16]:
res = list(get_all_books(session))
book_id, book_name = None, None
print(len(res))
for r in res:
    print(r.book_id, r.book_name, r.is_reserved)
    # if r.is_reserved:
    #     print(r.book_id, r.book_name, r.is_reserved)
    if r.book_name == "Book 1":
        book_id = r.book_id
        book_name = "Book 1"

book_id, book_name

2001
427e8338-f160-4c37-8628-0eda0dac2d2a Book 217 True
5fe9190a-1a08-4986-a5a1-31e81b1f3779 Book 1493 True
a178ece4-f228-486c-a61f-b03447f73df4 Book 1354 True
b088af79-213a-4d0c-809e-39356e0fed3d Book 1669 True
ec659d20-1f0f-46e7-a83c-2c502c6b872b Book 804 True
cb308802-5170-452b-9383-6d3755fa51ea Book 771 True
cc091a5e-68ce-40eb-8050-040b920889c1 Book 1837 True
5b89457f-0fb3-422f-b274-9580b843b85a Book 956 True
4ee28450-691e-4936-8260-86352d5ddb43 Book 107 True
2c4587f0-4357-4f47-ac20-96b7a008b626 Book 1991 True
1810a367-f5ca-4cbb-9ef3-ca045504a7e1 Book 1508 True
b7fdb197-f515-401b-ac69-543d94a8aa06 Book 1954 True
f2089ea8-c60b-49dd-9506-b2893df0ee67 Book 1533 True
f8cfc3b8-1064-4e8a-af87-f47bf4d02a9c Book 341 True
dee99f0a-2965-449b-af90-fa49eeed48db Book 1514 True
8e750f57-113d-4b6b-8e03-48d27282bc1b Book 1193 True
2aa6f9a5-d995-4dc9-a260-69db8d04590c Book 870 True
8910b36a-1ed8-460d-ba02-baffdb0a90a0 Book 982 True
0e81882e-4c51-4086-ba56-a381b01a7a81 Book 1126 True
5f8fcfc1-1b58-4

(UUID('e53e2750-dd76-4f9e-bab2-bc931bf734d3'), 'Book 1')

In [17]:
res = list(get_all_users(session))
for r in res:
    if r.user_name == "User 1":
        user_name = "User 1"
        user_id = r.user_id
    # if r.reservation_ids_list is not None:
    #     print(r.user_id, r.user_name, r.reservation_ids_list)
    print(r.user_id, r.user_name, r.reservation_ids_list)

user_name, user_id

0ebeb98c-dd33-44ff-88ac-7309a7b2020d User 24 None
6a6853a8-09bf-40fa-92ae-d45aec57fae5 User 80 None
2862c92a-b8e6-40c2-abcf-ae694ec9efaf User 76 [UUID('f37442aa-1077-4835-9df7-11ed25d4937e')]
c484516a-a715-4021-9e11-46c59c01b204 User 56 [UUID('90feb417-ab8e-4696-a1a4-e54afced43b8')]
55d955d4-28b3-4e03-baad-28a01c2b8f11 User 60 None
04baa876-94cf-4e0a-b90e-f214540fe5bf User 69 None
5df7293b-3076-4e4e-8850-c50cc59a38e8 User 98 None
8cdb4bc7-0c21-483d-b686-2610fd3557ce User 53 [UUID('cefd1d7f-1a42-466c-a498-9b3c869969a4')]
cf10d511-fdc7-43fd-8f46-cad92b8377a9 User 1 [UUID('9639832d-7b6c-43b7-a054-e8ede1d55190'), UUID('9ceecc2c-fcea-4a7b-8861-94e947110015'), UUID('f1bd0c0f-34dd-44e9-bd84-716681bf3651'), UUID('d843265d-6ef4-4dde-b37c-77828eabd256'), UUID('0b27aeaf-93a8-4fe5-bbf0-7759691a1dbe'), UUID('85ecc8d7-2634-42f9-96d7-e433ead69f0c'), UUID('28a1dd56-13d1-4421-9f98-98d7d8dc33ca'), UUID('07762819-fb0d-45c3-bd3f-d59a33206584'), UUID('da35db1f-3827-4ae1-a774-e8d05d16e7e9'), UUID('fa49319d-

('User 1', UUID('cf10d511-fdc7-43fd-8f46-cad92b8377a9'))

In [18]:
def test_reserve_cancel_debug(session, reservation_id, user_id, user_name, book_id, book_name, repeat_num=10_000):
    # Creates and cancels a reservation 10_000 times

    # for _ in range(repeat_num):
    for _ in range(3):
        add_reservation(session, reservation_id, user_id, user_name, book_name, book_id)

        print("Created")
        res = get_all_reservations(session)
        for r in res:
            if r.reservation_id == reservation_id:
                print(r.reservation_id, r.user_id, r.user_name, r.book_name)

        cancel_reservation(session, reservation_id)
        print("Cancelled")
        res = get_all_reservations(session)
        for r in res:
            if r.reservation_id == reservation_id:
                print(r.reservation_id, r.user_id, r.user_name, r.book_name)


def test_update_reservations_debug(session, num_reservations=1000):
    reservations = list(get_all_reservations(session))
    books = get_all_books(session)
    available_books = []
    for book in books:
        if not book.is_reserved:
            available_books.append(book)

    assert len(reservations) >= num_reservations, f"There should be at least a {num_reservations} existing reservations"
    assert len(available_books) >= num_reservations, f"There should be at least a {num_reservations} available books"

    for i in tqdm(range(num_reservations)):
        reservation = reservations[i]
        new_book = available_books[i]

        assert reservation.book_id != new_book.book_id
        assert reservation.book_name != new_book.book_name

        update_reservation_book(session, reservation.reservation_id, new_book.book_id)
        
        updated_reservation = get_reservation_by_id(session, reservation.reservation_id)
        
        assert updated_reservation.book_id == new_book.book_id
        assert updated_reservation.book_name == new_book.book_name

def test_make_random_requests_two_users_debug(user1, user2, num_repeats=10_000):
    # Simulates two users making 10_000 total random requests

    def make_random_requests(user_id, user_name, num_actions):
        # Simulates a user making random requests num_actions number of times

        user_cluster = Cluster(CLUSTER_IDS)
        user_session = user_cluster.connect(KEYSPACE)

        possible_requests = ["update_reservation_book", "update_reservation_user", "cancel_reservation", "make_reservation"]

        while num_actions > 0:

            selected_action = random.choice(possible_requests)

            # Update DB state information
            reservations = list(get_all_reservations(user_session))

            # If no reservations left, make a reservation
            if len(reservations) == 0:
                selected_action = "make_reservation"

            books = list(get_all_books(user_session))
            users = list(get_all_users(user_session))

            if selected_action == "update_reservation_book":
                reservation = random.choice(reservations)
                book = random.choice(books)

                if user_name == "User 1":
                    print(f"User1 Num actions left: {num_actions}")
                    
                else:
                    print(f"User2 Num actions left: {num_actions}")

                print("   Action:", selected_action)
                print("   Reservation:", reservation)
                print("   Book:", book)
                print("============================")
                    
                # Does not assume book is available
                update_reservation_book(user_session, reservation.reservation_id, book.book_id)

            elif selected_action == "update_reservation_user":
                reservation = random.choice(reservations)
                user = random.choice(users)
                
                if user_name == "User 1":
                    print(f"User1 Num actions left: {num_actions}")

                else:
                    print(f"User2 Num actions left: {num_actions}")

                print("   Action:", selected_action)
                print("   Reservation:", reservation)
                print("   User:", user)
                print("============================")

                # Does not assume user does not possess reservation already
                update_reservation_user(user_session, reservation.reservation_id, user.user_id)

            elif selected_action == "cancel_reservation":
                reservation = random.choice(reservations)
                
                if user_name == "User 1":
                    print(f"User1 Num actions left: {num_actions}")

                else:
                    print(f"User2 Num actions left: {num_actions}")
                
                print("   Action:", selected_action)
                print("   Reservation:", reservation)
                print("============================")

                cancel_reservation(user_session, reservation.reservation_id)

            elif selected_action == "make_reservation":
                available_books = [book for book in books if not book.is_reserved]

                if user_name == "User 1":
                    print(f"User1 Num actions left: {num_actions}")

                else:
                    print(f"User2 Num actions left: {num_actions}")

                print("   Action:", selected_action)
                print("   Available books:", len(available_books))

                # There are books available
                if len(available_books) > 0:
                    selected_random_book = random.choice(available_books)
                    print("   Selected book:", selected_random_book)

                    try:
                        add_reservation(user_session, uuid.uuid4(), user_id, user_name, selected_random_book.book_name, selected_random_book.book_id)
                    except:
                        pass
                    
                # No available books, dont use action
                else:
                    num_actions += 1

                print("============================")

            num_actions -= 1

        user_session.shutdown()

    thread1 = Thread(target=make_random_requests, args=[user1.user_id, user1.user_name, num_repeats//2])
    thread2 = Thread(target=make_random_requests, args=[user2.user_id, user2.user_name, num_repeats//2])

    thread1.start()
    thread2.start()

    thread1.join()
    thread2.join()

In [19]:
def test_same_request(session, reservation_id, new_book_id, repeat_num=10_000):
    # Runs an update of the book in a reservation 10_000 times

    for _ in tqdm(range(repeat_num)):
        update_reservation_book(session, reservation_id, new_book_id)


def test_reserve_cancel(session, reservation_id, user_id, user_name, book_id, book_name, repeat_num=10_000):
    # Creates and cancels a reservation 10_000 times

    for _ in tqdm(range(repeat_num)):
        add_reservation(session, reservation_id, user_id, user_name, book_name, book_id)
        cancel_reservation(session, reservation_id)

    add_reservation(session, reservation_id, user_id, user_name, book_name, book_id)


def test_update_reservations(session, num_reservations=1000):
    # Updates a 1000 reservations

    reservations = list(get_all_reservations(session))
    books = get_all_books(session)
    available_books = []
    for book in books:
        if not book.is_reserved:
            available_books.append(book)

    assert len(reservations) >= num_reservations, f"There should be at least a {num_reservations} existing reservations"
    assert len(available_books) >= num_reservations, f"There should be at least a {num_reservations} available books"

    for i in tqdm(range(num_reservations)):
        reservation = reservations[i]
        new_book = available_books[i]

        update_reservation_book(session, reservation.reservation_id, new_book.book_id)

# TODO: currently the same book can be reserved by two users.
# Add random wait time before checking availability/locks/idk
def test_make_all_reservations_two_users(user1, user2):
    # Simulates two users reserving all possible books

    def make_all_reservations(user_id, user_name):
        # Simulates user making all possible reservations one by one

        user_cluster = Cluster(CLUSTER_IDS)
        user_session = user_cluster.connect(KEYSPACE)

        # Update available book state
        books = get_all_books(user_session)
        available_books = [book for book in books if not book.is_reserved]
    
        while len(available_books) > 0:
            selected_random_book = random.choice(available_books)
            
            try:
                add_reservation(user_session, uuid.uuid4(), user_id, user_name, selected_random_book.book_name, selected_random_book.book_id)
            except:
                pass
            
            books = get_all_books(user_session)
            available_books = [book for book in books if not book.is_reserved]

        user_session.shutdown()

    thread1 = Thread(target=make_all_reservations, args=[user1.user_id, user1.user_name])
    thread2 = Thread(target=make_all_reservations, args=[user2.user_id, user2.user_name])

    thread1.start()
    thread2.start()

    thread1.join()
    thread2.join()


def test_make_random_requests_two_users(user1, user2, num_repeats=10_000):
    # Simulates two users making 10_000 total random requests

    def make_random_requests(user_id, user_name, num_actions):
        # Simulates a user making random requests num_actions number of times

        user_cluster = Cluster(CLUSTER_IDS)
        user_session = user_cluster.connect(KEYSPACE)

        possible_requests = ["update_reservation_book", "update_reservation_user", "cancel_reservation", "make_reservation"]

        while num_actions > 0:

            selected_action = random.choice(possible_requests)

            # Update DB state information
            reservations = list(get_all_reservations(user_session))

            # If no reservations left, make a reservation
            if len(reservations) == 0:
                selected_action = "make_reservation"

            books = list(get_all_books(user_session))
            users = list(get_all_users(user_session))

            if selected_action == "update_reservation_book":
                reservation = random.choice(reservations)
                book = random.choice(books)

                # Does not assume book is available
                update_reservation_book(user_session, reservation.reservation_id, book.book_id)

            elif selected_action == "update_reservation_user":
                reservation = random.choice(reservations)
                user = random.choice(users)

                # Does not assume user does not possess reservation already
                update_reservation_user(user_session, reservation.reservation_id, user.user_id)

            elif selected_action == "cancel_reservation":
                reservation = random.choice(reservations)
            
                cancel_reservation(user_session, reservation.reservation_id)

            elif selected_action == "make_reservation":
                available_books = [book for book in books if not book.is_reserved]

                # There are books available
                if len(available_books) > 0:
                    selected_random_book = random.choice(available_books)

                    try:
                        add_reservation(user_session, uuid.uuid4(), user_id, user_name, selected_random_book.book_name, selected_random_book.book_id)
                    except:
                        pass
                    
                # No available books, dont use action
                else:
                    num_actions += 1

            num_actions -= 1

        user_session.shutdown()

    thread1 = Thread(target=make_random_requests, args=[user1.user_id, user1.user_name, num_repeats//2])
    thread2 = Thread(target=make_random_requests, args=[user2.user_id, user2.user_name, num_repeats//2])

    thread1.start()
    thread2.start()

    thread1.join()
    thread2.join()

In [20]:
users = get_all_users(session)
user1, user2 = None, None
for user in users:
    if user.user_name == "User 1":
        user1 = user

    if user.user_name == "User 2":
        user2 = user

user1, user2

(Row(user_id=UUID('cf10d511-fdc7-43fd-8f46-cad92b8377a9'), reservation_ids_list=[UUID('9639832d-7b6c-43b7-a054-e8ede1d55190'), UUID('9ceecc2c-fcea-4a7b-8861-94e947110015'), UUID('f1bd0c0f-34dd-44e9-bd84-716681bf3651'), UUID('d843265d-6ef4-4dde-b37c-77828eabd256'), UUID('0b27aeaf-93a8-4fe5-bbf0-7759691a1dbe'), UUID('85ecc8d7-2634-42f9-96d7-e433ead69f0c'), UUID('28a1dd56-13d1-4421-9f98-98d7d8dc33ca'), UUID('07762819-fb0d-45c3-bd3f-d59a33206584'), UUID('da35db1f-3827-4ae1-a774-e8d05d16e7e9'), UUID('fa49319d-b2b4-4b94-a0df-c2c60c0315bd'), UUID('3dffdf68-c172-4f89-97aa-bffe333fd734'), UUID('f5c26146-19ac-4370-b35e-ab3574f470f1'), UUID('5cb29b06-7517-48c0-97f3-01ca5aa8d5cb'), UUID('715c7315-24f2-49c4-8595-f68777bd37e8'), UUID('249dd1fa-12d5-4e0a-af2d-e035d8370a50'), UUID('5831b158-0d98-4088-9e1f-ad8ee14b3324'), UUID('e93bc098-2f72-445f-b541-b2bede46bf78'), UUID('05bef6a5-9fe7-4075-b656-12b4bb4b0269'), UUID('5736dbd1-f7f2-44b4-b0b4-63737438404e'), UUID('ccf7538f-127b-4516-a0be-1c0c46a5f036'),

In [21]:
test_make_random_requests_two_users(user1, user2, 10_000)

In [22]:
test_make_all_reservations_two_users(user1, user2)

In [23]:
all_reservations = get_all_reservations(session)

user_1_reservations = 0
user_2_reservations = 0
for reservation in all_reservations:
    if reservation.user_id == user1.user_id:
        user_1_reservations += 1

    if reservation.user_id == user2.user_id:
        user_2_reservations += 1

print(f"User 1 reservation count: {user_1_reservations}")
print(f"User 2 reservation count: {user_2_reservations}")

User 1 reservation count: 565
User 2 reservation count: 570


In [24]:
res = list(get_all_reservations(session))
user_1_reservations = [r for r in res if r.user_id == user1.user_id]
user_2_reservations = [r for r in res if r.user_id == user2.user_id]

user_1 = get_user(session, user1.user_id)
user_2 = get_user(session, user2.user_id)

for r in user_1_reservations:
    if r in user_2_reservations:
        print("Error user1 reservation in user2 reservations list")

    if r.reservation_id not in user_1.reservation_ids_list:
        print("Error user1 reservation not in list")

for r in user_2_reservations:
    if r in user_1_reservations:
        print("Error user2 reservation in user1 reservations list")

    if r.reservation_id not in user_2.reservation_ids_list:
        print("Error user2 reservation not in list")

In [25]:
book_ids_1 = set([r.book_id for r in user_1_reservations])
book_ids_2 = set([r.book_id for r in user_2_reservations])

assert len(book_ids_1 & book_ids_2) == 0, "No book should be reserved by both clients at the same time"

AssertionError: No book should be reserved by both clients at the same time

In [ ]:
book_ids_1 & book_ids_2

{UUID('35f42768-c776-44b8-8792-bee48d439181'),
 UUID('90c86772-d5b2-4638-82c5-7203c2c58674'),
 UUID('a5158b5f-3b9d-43a2-b059-49aea66b7886'),
 UUID('baa8dda6-ce01-47be-a147-8b3cc55bc330')}

In [ ]:
book = list(book_ids_1 & book_ids_2)[0]
for res in get_all_reservations(session):
    if res.book_id == book:
        print(res.reservation_id, res.user_name, res.book_name, res.book_id)

eda0d700-677c-4df0-b291-6d3a82598039 User 1 Book 310 35f42768-c776-44b8-8792-bee48d439181
4fe17768-3b14-4e7b-8a66-142f6eb91b06 User 2 Book 310 35f42768-c776-44b8-8792-bee48d439181


In [ ]:
books = get_all_books(session)
available_books = [book for book in books if not book.is_reserved]
available_books

[]

In [ ]:
test_update_reservations(session)

100%|██████████| 1000/1000 [00:12<00:00, 77.19it/s]


In [ ]:
test_reserve_cancel(session, reservation_id, user_id, user_name, book_id, book_name)

res = get_all_reservations(session)
for r in res:
    print(r.reservation_id, r.user_id, r.user_name, r.book_name)
    

100%|██████████| 10000/10000 [03:33<00:00, 46.90it/s]


44c4f1be-d266-437b-9dcf-bbfe45eec299 a26c83c2-3a1b-4f26-b18c-a0a8385bc492 User 1 Book 1001


In [ ]:
test_same_request(session, reservation_id, book_id)

res = get_all_reservations(session)
for r in res:
    print(r.reservation_id, r.user_id, r.user_name, r.book_name)

100%|██████████| 10000/10000 [00:44<00:00, 226.46it/s]

44c4f1be-d266-437b-9dcf-bbfe45eec299 a26c83c2-3a1b-4f26-b18c-a0a8385bc492 User 1 Book 1


In [ ]:
session.shutdown()
cluster.shutdown()